# 

<img src= "https://lh3.googleusercontent.com/proxy/oQzNWc06hWUHGJFnv_TDAmr2dAdKjDp-xYzHlHqhw-Jp-tPN5jT3662f1-ocUzh1jXpG-oTKyZeiFGX-bewEnpDnmZFXMwAYtonepds_jy-rD2keO6dbb0hC7vmNrf3JQeGfbPVjAUs" alt ="class" style='width: 900px;'>


<a id="1"></a> <br>
# Introduction


This notebook is adressed to beginners who have just started their journey in machine learning, and having confused ideas about classification vocabulary in particular, in which i have tried to present typical data anomalies that you may face during your journey and what impact this may have on your model performance.

The first part is reserved to a synthetic dataset, so we can modify it as needed and generate different types of anomalies in order to understand basics of binary classification, afterwards we'll apply what we've learned to a real world dataset, for these task, i have chosen two widely popular models;

* Support vector machines
* Feed forward neural networks

these algorithms are already implemented in python ML libraries, and are ready to use, but before doing this, we first explain how each model get to learn from data, and what are the parameters of each one.  
  



📍📍📍 Please don't hesitate to let me know if you have any question, or need further clarification.

In [ ]:
# required import  

import keras  #this package must be installed first using your package manager
from keras.models import Sequential
from keras.layers import Dense

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb

import sklearn
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve, auc


In [ ]:
# a function that plots decision boundary and encircle support vectors of SVM algorithm

def bndry(x,y,mdl):
    
    plt.scatter(x[:, 0], x[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

# create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = mdl.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
    ax.scatter(mdl.support_vectors_[:, 0], mdl.support_vectors_[:, 1], s=100,linewidth=1, facecolors='none', edgecolors='k')
    plt.show()
    


# 1. Support vector machines

### The theory behind

let the training dataset   ${D}_{n} = \{({x}_{i},{y}_{j}) ,{i}=1,...,{n}\}$ with  ${x}_{i} \in R^{p}  , {y}_{i} \in \{-1, 1\} $

the objectif of the svm algorithm is to use ${D}_{n}$ to construct a function $ f:R^{p} \to R $ such that $sign(f)$ is a classifier, each new point $x$ will be classified in one of the two classes depending on the $sign$ of $f(x)$ .  

the parametric identification of $f$ relies on the minimization of the classification error, we have 3 cases here;


 ##### 1.1 Data is linearly separable (hard margin)
 
we assume that the positive (${y}_{i}=1$) and the negative (${y}_{i}= -1$) examples can be perfectly separated by a hyperplane ${H}_{0} $: {$x:f(x) = {\beta}_{0}+ {x}^{T}\beta $ } without error, where $\beta \in R^{p} $  is the weight vector and ${\beta}_{0} \in R $ is the intercept, in this case there is an infinity of such hyperplans.

 to choose the best one the idea here is to separate the two classes with a sufficient margin free of examples wich means find two parallel hyperplans ${H}_{-} and {H}_{+}$, each cross one or more data point (called support vectors) and the separating hyperplan $ H_0 $ is that one in the middle.  
Analitically, if the two classes are linearly separable there exist ${\beta}_{0}$ and  $\beta$ such that 
*  ${\beta}_{0} + {x}_{i}^{T}\beta \ge 1$  if ${y}_{i} = +1 $  
*  ${\beta}_{0} + {x}_{i}^{T}\beta \le -1$    if ${y}_{i} = -1 $      
    > $\beta $ and ${\beta}_{0}$ can always be rescaled by multiplying both by a positive number  

the support vectors ${x}_{*}$ are the data points satisfying $({\beta}_{0} - 1) + {x}_{*}^{T}\beta = 0$ and  $({\beta}_{0} + 1) + {x}_{*}^{T}\beta = 0$  

![Image](https://i.stack.imgur.com/CqGQC.png)

to find out the parameters we want to maximize the margin or the distance $d$ between the two hyperplans defined by their equations  
                    $ {H}_{-} : ({\beta}_{0} + 1) + {x}^{T}\beta = 0$  
                    $ {H}_{+} : ({\beta}_{0} - 1) + {x}^{T}\beta = 0$  

under the condition  $ {y}_{i}({\beta}_{0} + {x}_{i}^{T}\beta) \ge 1$ ,  $ i=1,...,n$  

we have $d=\frac2{||\beta||}$ and maximize $d$ return to minimize $\frac{{||\beta||}^2}2$  
so the problem can be formulated as following :  


<center> $ minimize$ $\frac{{||\beta||}^2}2$     </center>
   $\space    $    $ subject$  $to $    
 <center> $ {y}_{i}({\beta}_{0} + {x}_{i}^{T}\beta) \ge 1$ ,  $ i=1,...,n$ </center>

The optimization problem (OP) is convex (quadratic programm) ,then a global minimum exists and can be calculated using the QP solver implemented in [libsvm](https://www.csie.ntu.edu.tw/~cjlin/libsvm/) .  

let $\beta^*$ and $\beta_0^*$ be the solution of the OP, then the optimal hyperplane is given by $f^*(x)=0$ with $f^*(x)= \beta_0^* + x^T \beta^*$



 ##### 1.2 Data is linearly nonseparable (soft margin)
 
in real life application we can not expect a clear-cut separation between the two classes, instead because of noise, we can have overlapping classes, so some of the constraints related to the previous model will be violated, to have more flexible formulation of the problem we introduce nonnegative slack variables ${\xi}_{i}$ , one for each observation $({x}_{i},{y}_{i})$

the optimization problem become  



<center> $ minimize$  $\frac{{||\beta||}^2}2 +C\sum_{i} {\xi}_{i}  $</center>
       $ subject$  $to $    
<center> $ {y}_{i}({\beta}_{0} + {x}_{i}^{T}\beta) \ge 1-{\xi}_{i}$ ,  $ i=1,...,n$ </center>  
<center> $ {\xi}_{i} \ge 0$ </center>  

the idea here is to find an optimal hyperplane by maximizing the margin while softly penalizing misclassified points , thus for data points obeying the hard margin constraints ${\xi}_{i}=0$, the hyper-parameter $C$ is tunned by the user, for small values the margin becomes wide and tolerate more misclassifications. Generally $C$ is determined using validation set if datasize is huge enough, otherwise, using cross-validation technique.  

In this case the set of support vectors are defined by {$x^* :  {y}_{i}({\beta}_{0} + {x^*}^{T}\beta) - (1-{\xi}_{i})=0 $}   

 

 ##### 1.3 Nonlinear SVM
 
in some cases (actually the more realisitic ones) data classes are entirly overlapped and there is no way to separate them by an hyperplan despite allowing some classification errors, to handle this problem, we use a nonlinear mapping $ \Phi : {R}^{p} \to \mathcal{H}  $ to map each observation ${x}_{i} \in {R}^{p}$ into a higher dimensional feature space $\mathcal{H}$ ;  

$\mathcal{H}$ will be assumed to have the structure of a Hilbert space of real valued functions on ${R}^{p}$ with scalar product and norm.  
**the fantastic idea of nonlinear SVM is that the separation of the classes by means of an optimal separating hyperplan will be easier in that new feature space**.  
The transformed sample is {$\Phi({x}_{i}),{y}_{i}$}, and can be used to find the separating hyperplan (same previous procedure), but while resolving the optimization problem, the term  $ \Phi({x}_{i})^{T}\Phi({x}_{j})$ may be very hard to compute and sometimes impossible in the case of $\mathcal{H}$ infinite-dim
this difficulty can be avoided by the kernel trick, it consists in computing the scalar product by means a nonlinear kernel function $K({x}_{i},{x}_{j}) = \Phi({x}_{i})^{T}\Phi({x}_{j})$ defined in input space without even knowing the analytical form of $\Phi$.  

examples of popular kernels  
* linear kernel : $K({x}_{i},{x}_{j}) = 1+{x}_{i}^{T}{x}_{j}$  
  
  
* polynomial kernel : $K({x}_{i},{x}_{j}) = ({1+{x}_{i}^{T}{x}_{j} })^{d}$  
  
  
* gaussian kernel : $K({x}_{i},{x}_{j}) = \exp(-{||{x}_{i}-{x}_{j}||}^2/2{\sigma}^2)$

the hyperparamers of the kernel functions like $d$ and $\sigma$ are tunned using validation set or cross validation technique



# 2. Feedforward neural network


feedforward neural network (FNN) is an artificial neural network having a layered architecture, the first and latest are called the input and output layers respectively, the interior layers are called hidden layers.  
each layer is composed of one or more nodes, and each node is connected with all nodes existed in the previous and following layer by real valued weights, but not to nodes in the same layer. FNN are generally implemented with an additional node in each layer, called the bias unit, except the output layer. This node plays the role of the intercept term ${\beta}_{0}$ in linear models.  
the interest of this architecture is the capacity of handling non-linearly separable data and huge datasize

let

* $p$ the number of inputs or features,
* $L$ the number of layers,
* ${H}^{(l)}$ the number of nodes of the lth layer $(l = 1,...,L)$ of the FNN, 
* ${w}^{(l)}_{kv}$ denote the weight of the link connecting the $k$th node in the $l−1$ layer and the $v$th node in the $l$ layer, 
* ${z}^{(l)}_{v}$ , $v = 1,...,{H}^{(l)}$ the output of the $v$th hidden node of the lth layer, 
* ${z}^{(l)}_{0}$ denote the bias for the $l, l = 1,...,L$ layer. 
* Let ${H}^{(0)}=p$ and ${z}^{(0)}_{v}={x}_{v}, v = 0,...,p.$  


 the output of the $v$th node of the $l$th layer, is  gived by 
 
 
 
 <center> ${z}^{(l)}_{v} = {g}^{(l)}(\sum\limits_{k=0}^{{H}^{(l-1)}}  {w}^{(l)}_{kv} {z}^{(l-1)}_{k} ) $,     $v = 1,...,{H}^{(l)}$</center>    
 
 
${g}^{(l)}(.)$ is a non linear transformation called the activation function, here we cite the most commonly used for binary classification 

  

\begin{align*}
g(a) = \frac{1}{1+e^{-a}} && \text{Logistic sigmoid} \\
g(a) = \tanh(a) && \text{Hyperbolic tangent} \\
g(a) = 
\begin{cases}
0 & a < 0 \\
a & a \ge 0 \\
\end{cases} && \text{ReLU (Rectified Linear Unit)} \\
\end{align*}


The set of weights is constructed while the training procedure using the backpropagation algorithm, and the number of hidden nodes $H$ and hidden layers reflects the complexity of the architecture, the greater the number the more complex the architecture. usually they are fixed using cross-validation.
        
    
### Backpropagation  


once the number of hidden nodes is given, the weights are calculted in an iterative way basing on the training set ${D}_{n}$, It is a gradient-based algorithm that aims to minimize the loss function  
we notice that the loss function may change according to the learning task, for example, for a binary classification problem, Cross-entropy is the most popular 

<center> $ C = L_{cross-entropy}(\hat y, y) = - \sum\limits_{i=1}^n {y}_{i}\log  ({\hat y}_{i})$   </center>

weights take random initial (small) values , and with each observation $({x}_{i},{y}_{j})$ these values are updated  

<center> $W(t)= W(t-1) - \eta \frac{\partial C}{\partial W(t-1) }$ </center>

    
> $W(t)$ refers to the weight vector  {${w}^{(l)}_{kv}$}  in iteration t  
$\eta$ is an hyperparameter called the learning rate  


like hyperparameters of SVM algorithm, $\eta$ can be tunned using validation set or cross validation
       




# 3.Application on synthetic data 

 in this section we will focus on the impact of some data problems on the algorithm performance, to do so we generate small data sizes with 2 features to be able to make clear visualisations.

### data generation

we will use make_classification function from sklearn.datasets library to generate a random 2-class classification problem. This function allows controlling the level of noisiness and a lot of bias generators like outliers and class imbalance by tuning its parameters, for example

* weights: control the proportions of samples assigned to each class
* flip_y: the degree of noisiness, the higher the value the harder is the classification task, wich means for large values the data can not be linearly separable and need nonlinear methods
* class_sep: control the distance between classes, for small values classes are overlaped
 
### simple case  
> In first place we, we treat the simplest case, we tune the parameters to have a linearly separable data classes

In [ ]:
X,Y= make_classification(n_samples=100, n_features=2,n_informative=2, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.5]),\
                        flip_y=0.001, class_sep=2,random_state=69)
# show the database's head
# df=pd.DataFrame(np.append(x, y.reshape(100,1), axis=1),columns=["v"+str(i) for i in range(1,4)])
df = pd.DataFrame(dict(v1=X[:,0], v2=X[:,1], target=Y))
df.head()   # show how our data looks

In [ ]:
sb.lmplot("v1","v2",df,"target",fit_reg=False) #scatter plot of our data

##### training
To train the model we split the data set to 2 parts : 70% for training set, and 30% for testing, 
the validation techniques will be employed in the next section when we use real world dataset with size



the test set allows the calculation of an unbiased estimator of the outsample error $\hat E_{out} = \frac{FP+FN}{TP+TN+FP+FN}$   
besides,several metrics based on the confusion matrix can be calculated in order to measure the model performance  

* accuracy = $\frac {TP+TN}{TP+TN+FP+FN}$    
  

* precision = $\frac {TP}{TP+FP}$    
    
    
* recall = $\frac {TP}{TP+FN}$      
  
  
* f1score = $\frac {2*precision*recall}{precision + recall}$

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=42) 

In [ ]:
mdl=svm.SVC(kernel='linear')  # choose the linear kernel, C is set on default value
mdl.fit(X_train,Y_train)          # fit the model on the training set
predictions =mdl.predict(X_test)    # predict the labels of x_test

bndry(X_train,Y_train,mdl)          # plot the decision function 

In [ ]:
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))
print("the outsample accuracy = ", accuracy_score(Y_test,predictions))
print("the insample accuracy = ", accuracy_score(Y_train,mdl.predict(X_train)))

the overall accuracy reached is 100% and the outsample error $\hat E_{out}  = 0$, the algorithm can perfectly generalize 

## 3.1 Impact of mislabeled training examples on the SVM performances

as we have seen in theory section, the creation of the decision boundary relies on support vectors, which means that the labels of these vectors are very important and are responsible for the performane of the algorithm, 
in case of the existence of mislabeled data points, there is a big chance that these data points will be nominated as support vectors since they are borderline examples.  
to see how this really impact the performance, we modify some examples label  in the training set 


In [ ]:
X,Y= make_classification(n_samples=100, n_features=2,n_informative=2, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.5]),\
                        flip_y=0.4, class_sep=1.5,random_state=69)
df = pd.DataFrame(dict(v1=X[:,0], v2=X[:,1], target=Y))
sb.lmplot("v1","v2",df,"target",fit_reg=False)

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=42) 
mdl=svm.SVC(C=1,kernel='linear')
mdl.fit(X_train,Y_train)
predictions =mdl.predict(X_test)   

In [ ]:
bndry(X_train,Y_train,mdl) 

support vectors are those points encircled, as we can see all of noisy examples are captured as support vectors, this means that the existence of mislabeled data increases the number of support vectors.   
a high number of support vectors makes the model more complex and computationally expensive; that because, in order to make prediction on a new data point, the model need to compute the dot product of each support vector with the test point.  
  
the following figure gives a quantification of the impact of outliers on SVM performances, we plot the number of support vectors vs the model outsample accuracy and insample accuracy;

In [ ]:
def classify(o,sep,w):
    X,Y= make_classification(n_samples=1000, n_features=2,n_informative=2, n_redundant=0, n_clusters_per_class=1,weights=w,\
                        flip_y=o, class_sep=sep,random_state=69)
    X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=42) 
    mdl=svm.SVC(C=1,kernel='linear')
    mdl.fit(X_train,Y_train)
    predictions =mdl.predict(X_test)
    numberSV=mdl.n_support_.sum()
    out_acc=accuracy_score(Y_test,predictions)
    in_acc=accuracy_score(Y_train,mdl.predict(X_train))
    return numberSV,out_acc,in_acc

trn=[]
tst=[]
nSV=[]
l=[0.01, 0.1, 0.4, 0.7, 1]
for o in l:
    clf=classify(o,1,np.array([0.5]))
    nSV.append(clf[0])
    tst.append(clf[1])
    trn.append(clf[2])


fig, ax = plt.subplots(figsize=(10,7))
ax2 = ax.twiny()
ax2.set_xlabel(r"proprtion of mislabeled data")
ax.plot(nSV,trn , label='train')
ax.plot(nSV,tst, label='test')
ax.set_xlabel(r"number of SV")
ax.set_ylabel(r"accuracy")
ax.legend(loc="upper right", frameon=False)
plt.show()

With a clean data (no mislabels or outliers),the number of support vectors is very small, in this case the model shows high performances on trainset as well as on testset, but with an increasing proportion of mislabeled data , the number of support vectors increases as well and the performance decreases in both sets.
Returning to our example, the model performs very badly on the test set (only 76% are correctly classified) even with different values of C

In [ ]:
print(accuracy_score(Y_test,predictions))
print(recall_score(Y_test,predictions))
print(precision_score(Y_test,predictions))

In [ ]:
for c in [0.1, 10, 100 , 10000]:
    mdl=svm.SVC(C=c,kernel='linear')
    mdl.fit(X_train,Y_train)
    predictions =mdl.predict(X_test) 
    print("C= ",c,", accuracy= ", accuracy_score(Y_test,predictions) )

## 3.2 Impact of class imbalance

to simulate the case of imbalanced data classes, we generate a sample where class 0 represent 90% of the data and class 1 represent 10%.

In [ ]:
X,Y= make_classification(n_samples=700, n_features=2,n_informative=2, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.9]),flip_y=0.001, class_sep=1.5,random_state=67) 
df = pd.DataFrame(dict(v1=X[:,0], v2=X[:,1], target=Y)) 
sb.lmplot("v1","v2",df,"target",fit_reg=False)

The C parameter is used as a penalty during the fit of the model, i.e the finding of the decision boundary. By default, each class has the same weighting, and the model try to learn from the two data classes in a balanced way.  
Now given that there are significantly more examples in the majority class than the minority class, this means that there are no enough data belonging to the minority class to learn from, and the classifier will favor the majority class. as we see in the figure below, the decision boundary is skewed to the left (towards the minority class); 
Another point is that some metrics can give misleading results in term of outsample performances, for example if the classifier in our example classify all new samples in class 0 (the majority class) the overall accuracy will be 90%, that's why in such cases of high imbalance level it's recomended to examine other metrics like precision due to its sensitivity to class imbalance because it considers the number of negative samples incorrectly labeled as positive. But precision alone is insufficient because it provides no insight into the number of samples from the positive group that were mislabeled as negative, On the other hand, Recall should be taken into account because it is only dependent on the positive group.


In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=48) 
mdl=svm.SVC(C=1,kernel='linear')
mdl.fit(X_train,Y_train)
predictions =mdl.predict(X_test)  
bndry(X_train,Y_train,mdl) 

In [ ]:
print(confusion_matrix(Y_test,predictions))
print(accuracy_score(Y_test,predictions))
print(recall_score(Y_test,predictions))
print(precision_score(Y_test,predictions))


the impact of classes imbalance on support vectors is not really remarkable, if the datasize is reasonably large which means having enough information about the minority class , there will be a very little eﬀect on the classiﬁer performance, and this's true since classes don't overlap, otherwise the classification task will become harder.  
In the following we examine the performance metrics values while increasing the data size

In [ ]:
for s in [3000, 5000, 15000 , 30000, 45000]:
    X,Y= make_classification(n_samples=s, n_features=2,n_informative=2, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.9]),flip_y=0.001, class_sep=1.5,random_state=67) 
    X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=23) 
    mdl=svm.SVC(C=1,kernel='linear')
    mdl.fit(X_train,Y_train)
    predictions =mdl.predict(X_test) 
    print("size= ",s,", accuracy= ", accuracy_score(Y_test,predictions)," ,precision= ", precision_score(Y_test,predictions)," ,recall= ",recall_score(Y_test,predictions)  )

## 3.3 Class overlap

in the theory section we've talked about the situation when data classes are overlapped and don't accept linear separation, let's see how this data looks and how the kernel trick can solve the problem

In [ ]:
X,Y= make_circles(n_samples=500,noise=0.2, random_state=0,factor=0.4)
df = pd.DataFrame(dict(v1=X[:,0], v2=X[:,1], target=Y))
sb.lmplot("v1","v2",df,"target",fit_reg=False)

we can clearly notice the non linearity of the separating hyperplane , so by means the kernel SVM method we'll try to determine which kernel function can construct the best classifier in terms of outsample accuracy

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=23) 

for k in ["rbf","poly", "sigmoid"]:
    mdl=svm.SVC(C=10,kernel=k)
    mdl.fit(X_train,Y_train)
    predictions =mdl.predict(X_test)
    
    print("{} kernel,   accuracy = {}".format(k,accuracy_score(Y_test,predictions)))
    bndry(X_train,Y_train,mdl)


among the kernel functions, the rbf (gaussian) kernel gives a clear separation cut, with an outsample accuracy of 94%.  
  

## 3.4 Feedforward neural network : impact of class imbalance on FNN performance

like all artificial neural networks, FNN needs reasonably large data to learn and be able to generalize, by means the make_classification function we generate binary classification task of size 5000 and 8 features, with only 10% of positive class

In [ ]:
x,y= make_classification(n_samples=5000, n_features=8,n_informative=8, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.9]),flip_y=0.2, class_sep=1,random_state=34)


In [ ]:
#prepare the train and test data
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=42)


To construct the FNN model we use keras's sequential model data structure, this tool helps in building up a stack of linear layers one bye one.

In [ ]:

# instanciate  sequantial model
model = Sequential()  
# build up the layers one by one
model.add(Dense(8, activation='relu', input_shape=(8,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

the two first layers contains 8 nodes each and use ReLu activation function, since features are numerical real numbers within ranges, the output is binary hence the last layer is a sigmoid output layer with one node , the input_shape indicate the number of features.  
  
Now, after creating model we need to call its compile() method to specify the loss function and optimizer to use, and it's ready to train !  
  
* Loss : indicates the loss function to be minimized while learning the weights
* optimizer : the algorithm used to minimize the loss function
* metrics : the metric that measures the model insample performance
* epoch : number of iteration (passes through the training dataset.)
* batch_size : when specified, the data is divided into equal-sized batchs and processed in parallel, so 1 epoch= #of batches

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
history=model.fit(x_train,y_train,epochs=20,batch_size=32, validation_data=(x_test, y_test))

In [ ]:
# predictions on test data.
y_pred = model.predict_classes(x_test)
print(confusion_matrix(y_test,y_pred))
print("accuracy = ",accuracy_score(y_test,y_pred))
print("precision = ",precision_score(y_test,y_pred))
print("recall = ",recall_score(y_test,y_pred))

the recall score is dramatically low (0.45) even if precision score is high, the number of false negative is relatively considerable compared to the number of false positive.  
in the next plot, we see how the recall metric (outsample and insample evaluation) decreases with high level of class imbalance.

In [ ]:
def gen(o):
    x,y= make_classification(n_samples=5000, n_features=8,n_informative=8, n_redundant=0, n_clusters_per_class=1,weights=np.array([o]),flip_y=0.2, class_sep=1,random_state=34)
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=42)
    model = Sequential()  
    model.add(Dense(8, activation='relu', input_shape=(8,)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
    model.fit(x_train,y_train,epochs=20,batch_size=32,verbose=0)
    y_pred = model.predict_classes(x_test)
    return recall_score(y_test,y_pred),recall_score(y_train, model.predict_classes(x_train))
trn=[]
tst=[]
l=[0.99, 0.9, 0.8, 0.7, 0.6,0.5]
for o in l:
    clf=gen(o)
    tst.append(clf[0])
    trn.append(clf[1])


fig, ax = plt.subplots(figsize=(10,7))
ax.set_xlabel(r"proprtion of mislabeled data")
ax.plot(l,trn , label='train')
ax.plot(l,tst, label='test')
ax.set_xlabel(r"proprtion of negative class in the dataset")
ax.set_ylabel(r"Recall")
ax.legend(loc="upper right", frameon=False)
plt.show()

class imbalance is a serious problem that impacts negatively any classification model performance if it's combined with other problems like existence of outliers and class overlap, it usually needs to be cured before the training step, there are many technique to do that, but not always work in case of severe level of imbalance.
in the next section when we'll use realworld dataset, we'll try to figure out some of these techniques

## 3.5 Impact of noisy data  on FNN performance

we start by generating a noisy sample with many mislabeled data and class overlap and then instantiate an FNN model.

In [ ]:

x,y= make_classification(n_samples=5000, n_features=8,n_informative=8, n_redundant=0, n_clusters_per_class=1,weights=np.array([0.5]),flip_y=0.8, class_sep=0.5,random_state=20)
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=89)

model = Sequential()  
model.add(Dense(8, activation='relu', input_shape=(8,)))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train,y_train,epochs=100,batch_size=32,verbose=0)
y_pred = model.predict_classes(x_test)


In [ ]:

print(confusion_matrix(y_test,y_pred))
print("outsample accuracy = " ,accuracy_score(y_test,y_pred))
print("insample accuracy = " ,accuracy_score(y_train, model.predict_classes(x_train)))
print("outsample recall = " ,recall_score(y_test,y_pred))
print("insample recall = " ,recall_score(y_train,model.predict_classes(x_train)))

Noise in dataset leads to poor accuracy of the classifier , adding more layers and nodes to the model or even a high number of epochs (100 epochs in the example) would probably improve the insample performance, but this leads to overfit and incapacity of generalization on unseen data, this explains the difference between the outsample metrics and insample metrics.


# 4. Real world dataset 

In this section  we will use "Electrical Grid Stability Simulated Dataset" hosted in University of California (UCI) Machine Learning [Repository](https://archive.ics.uci.edu/ml/index.php).  this dataset of dimendion 10000 contains 12 features and two target variables, "stab" variable for regression tasks which is unuseful for our case and "stabf" for classification task.
the purpose is to construct a model able to classify a DSGC as stable or unstable basing on 3 metrics measured for 4 participants to the DSGC ( one )  
* tau[x] : reaction time of participant x, x=1,2,3,4
* p[x] : nominal power produced or consumed by participant x, x=1,2,3,4
* g[x] : price elasticity coefficient for each network participant x, x=1,2,3,4

Decentral Smart Grid Control (DSGC) is an intelligent electricity network able to match electricity demand and supply decentrally and on an entirely self-organised basis, to prevent the hacker attacks in one side and ensure a constant supply independant of any disturbing factors in the other side.  
to evaluate the efficiency of such grid in real life, the physicists developed a mathematical model in which they simulated a 4-node star architecture, comprising one power source (a centralized generation node) supplying energy to three consumption nodes (the participants), the model takes into consideration inputs (features) described above.   


The dataset has a synthetic nature so there is no mislabeled samples; at first step, we begin by an exploratory analysis, afterwards, we'll try to construct two predictive models using SVM and FNN, and compare their performance. 

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv"
# import dataset directly from the repository to pandas dataframe
data = pd.read_csv(url)

# drop out the stab variable
data.drop("stab",axis=1,inplace=True) 

#encode the target to dummy variable
data['stabf'] = data['stabf'].replace({'unstable': 0, 'stable': 1}) 
data.head()

In [ ]:
      
#to verify the correlation between variables we display the correlation matrix
plt.figure(figsize=(11,5))
sb.heatmap(data.iloc[:,:-1].corr(), annot=True)
plt.show()

the matrix reveals the existence of correlation between 'p1' and 'p2', 'p3', 'p4', as expected, because the total power consumed equals the total power generated, p1 (supplier node) = - (p2 + p3 + p4);  so p1 doesn't add any information and will be dropped out;


In [ ]:
data.drop("p1",axis=1,inplace=True) 

In [ ]:
print(sb.countplot(x="stabf", data=data))
print("proportion of each class")
print(data['stabf'].value_counts(normalize=True))

class 1 represents only 36% of the total dataset, to handle this imbalance, first we use "stratify" argument in "train_test_split" sickitlearn's method to preserve the data distribution after split, second, in the training step, we'll use class weight in loss function to penalize mistakes on the minority class;


In [ ]:
# data split
x = data.iloc[:, :10]
y = data.iloc[:, 11]
x_train, x_test, y_train, y_test=train_test_split(x,y, stratify=y,test_size=0.2,random_state=42) 

print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

in order to speed up the calculations and avoid the domination of features with large spread of values, we'll use standardization method of sickitlearn to rescale the feature space to be on the same scale.

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## SVM model

a great way to find out the hyperparameters and kernel that fit the best our training set, is to conduct a grid search using scikit-learn’s GridSearchCV.
  
> !!! the run time takes about 4 min !!!

In [ ]:
# instantiate an SVM model
model = svm.SVC()

#candidate values of hyperparameters
parameters = [{'C' : [1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001],'kernel': ['rbf']},
             {'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
             
grid = GridSearchCV(model, parameters, cv=10, n_jobs=-1)
grid.fit(x_train, y_train)

In [ ]:
print('Best C:',grid.best_estimator_.C) 
print('Best Kernel:',grid.best_estimator_.kernel)
print('Best Gamma:',grid.best_estimator_.gamma)

In [ ]:
model=svm.SVC(C=1,kernel='rbf',gamma=0.1,probability=True,class_weight='balanced')
model.fit(x_train,y_train)
y_pred =model.predict(x_test)
y_probas=model.predict_proba(x_test)[::,1]

print(confusion_matrix(y_test,y_pred))
print("outsample accuracy = " ,accuracy_score(y_test,y_pred))
print("insample accuracy = " ,accuracy_score(y_train, model.predict(x_train)))
print("outsample recall = " ,recall_score(y_test,y_pred))
print("insample recall = " ,recall_score(y_train,model.predict(x_train)))



The difference between the outsample and insample scores is inevitable given hyperparameter tuning, making the training scores optimistic, but in general the scores are close which prooves that the model is probably not overfit or underfit

# FNN model

FNN have a lot of hyperparameters to tune (from the external architecture to the hidden parameters of each function (ex:optimizer)), so it's very hard (may be impossible) to cross-validate every single value candidate. that's why i have tried several combinations (not all possible ones) and pick out the one that led to the best prediction performance on the test set.

In [ ]:
# Instantiate the model
classifier = Sequential()

# Input layer and first hidden layer
classifier.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))

# Second hidden layer
classifier.add(Dense(units = 10,kernel_initializer = 'uniform', activation = 'relu'))
# Single-node output layer
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# FNN compilation
opt = keras.optimizers.SGD(learning_rate=0.1)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
history=classifier.fit(x_train, y_train, epochs=35, verbose=0,validation_data=(x_test,y_test))
y_predfnn = classifier.predict_classes(x_test)
y_probas_nn=classifier.predict(x_test)

print("outsample accuracy = ",accuracy_score(y_test,y_predfnn))
print("insample accuracy = ",accuracy_score(y_train,classifier.predict_classes(x_train)))

In [ ]:
print(confusion_matrix(y_test,y_predfnn))


In [ ]:
# evaluate the model
train_acc = classifier.evaluate(x_train, y_train, verbose=0)
test_acc = classifier.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc[1], test_acc[1]))
# plot loss during training
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

The difference between the outsample scores and insample scores is not really considerable, they are almostly equal, we can say that the model gets to learn correctly and is not overfit or underfit.
   
> NB : if you run the model, your results may vary given the stochastic nature of the training procedure

# Comparaison using ROC curve

In order to visualize the performance of the classifiers and compare them we use ROC curve. It is a representation of true positive rate (TPR) as a function of the false-positive rate (FPR), And it’s useful only when compared with other curves  
This representation puts forward a new indicator called the area under the curve (AUC). The closer it gets to 1, the better is the classifier, in other words, the larger the air under the ROC curve of a model, the better this model is. 
a random classifier will have an equivalent ROC curve to the blue dashed line.   
as we see, both classifiers show aproximately the same performance (same auc), with a slight difference at the inflection point. 

In [ ]:
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test,  y_probas)

fpr_nn, tpr_nn, _ = metrics.roc_curve(y_test,  y_probas_nn)

auc = metrics.roc_auc_score(y_test, y_probas)

auc_nn = metrics.roc_auc_score(y_test, y_probas_nn)

plt.plot(fpr,tpr,label="SVM, auc="+str(auc))
plt.plot([0.0, 1.0],'b--',label="random guess model")
plt.plot(fpr_nn,tpr_nn,label="FNN, auc="+str(auc_nn))

plt.legend(loc=4)
plt.xlabel(r"False Positive Rate")
plt.ylabel(r"True Positive Rate")
plt.title('ROC curves')
plt.show()


# To Be Continued...